## 줄거리 기반 추천

In [ ]:
import pandas as pd
import numpy as np

#### 정기 연재 웹툰 정보

In [ ]:
daily = pd.read_csv('daily.csv')
daily

#### 비정기 연재 웹툰 정보

In [ ]:
dailyPlus = pd.read_csv('dailyPlus.csv')
dailyPlus

#### CSV 파일 통합 후 중복 제거 후 DataFrame 생성

In [ ]:
df = pd.concat([daily, dailyPlus], ignore_index=True)
df = df.drop_duplicates(subset='title', keep='first')

# DataFrame
df = df.set_index('id')
df

### 한국어 자연어 처리 - 형태소 토큰화

In [ ]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenizer_Kkma(text):
    
    # 내가 원하는 품사 (첫 알파벳)
    my_tag = ("N", "V", "M", "XR")
    
    # 형태소 분석기 정의
    kkma = Kkma()
    
    # 형태소 분석하기
    words_with_tag = kkma.pos(text)
    
    # 조건에 맞는 단어만 남겨놓기
    words = [word for word, tag in words_with_tag if (len(word) > 1) and (tag.startswith(my_tag))]
    
    return words

def tokenizer_Okt(text):
    
    # 내가 원하는 품사 - 명사, 동사, 부사, 형용사
    my_tag = ("Noun", "Verb", "Adverb", "Adjective")
    
    # 형태소 분석기 정의
    okt = Okt()
    
    # 형태소 분석하기
    words_with_tag = okt.pos(text)
    
    # 조건에 맞는 단어 남겨놓기
    words = [word for word, tag in words_with_tag if (len(word) > 1) and (tag.startswith(my_tag))]
    
    return words
    
# TF-IDF 벡터라이저 정의
# tfidf = TfidfVectorizer(tokenizer=tokenizer_Kkma, min_df=1)
tfidf = TfidfVectorizer(tokenizer=tokenizer_Okt, min_df=1)

# 벡터화하기
tfidf_matrix = tfidf.fit_transform(df['story'])

###  형태소 분석기 테스트

In [ ]:
text = "혼자서 자전거를 즐겨타던 모범생 조자현. 원치 않게 자전거 크루의 일에 자꾸 휘말리게 되는데... 자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마!"

# 꼬꼬마 형태소 분석기 정의
kkma = Kkma()
    
# 형태소 분석하기
words_with_tag = kkma.pos(text)

print(type(words_with_tag))
print(words_with_tag)

In [ ]:
text = "혼자서 자전거를 즐겨타던 모범생 조자현. 원치 않게 자전거 크루의 일에 자꾸 휘말리게 되는데... 자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마!"

# Open Korea Text 형태소 분석기 정의
okt = Okt()
    
# 형태소 분석하기
words_with_tag = okt.pos(text)

print(type(words_with_tag))
print(words_with_tag)

In [ ]:
tfidf_matrix.shape

In [ ]:
tfidf_matrix

### 코사인 유사도를 통한 단어 유사도 파악하기

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# linear_kernel : cosine_similarity와 결과는 같지만 속도는 더 빠름
# tfidf_matrix 벡터들간에 대한 내적을 통해 코사인 값을 비교
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

### 웹툰의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환

In [ ]:
# 웹툰 제목에 따른 인덱스 값 불러오는 리스트
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

In [ ]:
indices['호랑이형님']

In [ ]:
df.iloc[[461]]

In [ ]:
indices['캐슬2:만인지상']

In [ ]:
df.iloc[[193]]

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    # 웹툰 제목을 통해서 전체 데이터 기준 그 웹툰의 index 값을 얻기
    idx = indices[title]
    
    # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 코사인 유사도 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 자기 자신을 제외한 10개의 추천 엡툰을 슬라이싱
    sim_scores = sim_scores[1:21]
    
    # 추천 웹툰 목록 10개의 인덱스 정보 추출
    webtoon_indices = [i[0] for i in sim_scores]
    
    # 인덱스 정보를 통해 웹툰 제목 추출
    return df['title'].iloc[webtoon_indices]

In [ ]:
get_recommendations('신의 탑')

In [ ]:
get_recommendations('호랑이형님')

### 객체 파일로 저장

#### 추천 프로그램에서 사용하기 위한 [인덱스, 제목, 썸네일 주소, 웹툰 주소] 리스트만 가져오기

In [ ]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])
indices

In [ ]:
webtoon = df[['id', 'title', 'thumbnail_url', 'webtoon_url']].copy().set_index('id')
webtoon.head()

In [ ]:
import pickle

pickle.dump(webtoon, open('webtoon.pickle', 'wb'))
pickle.dump(cosine_sim, open('cosine_sim_pickle', 'wb'))